In [1]:
pip install pyspark

In [2]:
pip install pandas

In [3]:
pip install matplotlib

In [4]:
pip install requests

In [5]:
pip install numpy

In [6]:
import zipfile
import pandas as pd
from google.colab import files

# Função para carregar o arquivo CSV
def carregar_arquivo():
    # upload  ZIP
    uploaded = files.upload()

    # Descompactar o  ZIP
    for filename in uploaded.keys():
        if filename.endswith(".zip"):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall()
            print(f"Arquivo {filename} descompactado com sucesso!")

    # Ler o arquivo CSV descompactado
    csv_filename = "State_of_data_BR_2023_Kaggle - df_survey_2023.csv"  # Ajuste com o nome correto do arquivo descompactado
    try:
        df_pandas_csv = pd.read_csv(csv_filename, encoding='utf-8')  # Tentativa com UTF-8
        print("Arquivo CSV lido com sucesso com UTF-8!")
    except UnicodeDecodeError:
        df_pandas_csv = pd.read_csv(csv_filename, encoding='ISO-8859-1')  # Se UTF-8 falhar, tenta ISO-8859-1
        print("Arquivo CSV lido com sucesso com ISO-8859-1!")

    return df_pandas_csv

# Executar a função de upload e carregar os dados CSV no Pandas
df_pandas_csv = carregar_arquivo()

# Exibir os primeiros registros do DataFrame Pandas
# print("Dados no DataFrame Pandas:")
# print(df_pandas_csv.head())


Saving State_of_data_BR_2023_Kaggle - df_survey_2023.csv.zip to State_of_data_BR_2023_Kaggle - df_survey_2023.csv.zip
Arquivo State_of_data_BR_2023_Kaggle - df_survey_2023.csv.zip descompactado com sucesso!
Arquivo CSV lido com sucesso com UTF-8!


In [8]:

from pyspark.sql import SparkSession


# Dicionário para renomear as colunas
colunas_renomeadas = {
"('P0', 'id')": 'id',
"('P1_a ', 'Idade')": 'dsc_idade',
"('P1_a_1 ', 'Faixa idade')": 'dsc_faixa_idade',
"('P1_b ', 'Genero')": 'dsc_genero',
"('P1_c ', 'Cor/raca/etnia')": 'dsc_cor_raca_etnia',
"('P1_d ', 'PCD')": 'dsc_pcd',
"('P1_e ', 'experiencia_profissional_prejudicada')": 'dsc_experiencia_profissional_prejudicada',
"('P1_e_1 ', 'Não acredito que minha experiência profissional seja afetada')": 'flg_experiencia_profissional_nao_prejudicada',
"('P1_e_2 ', 'Experiencia prejudicada devido a minha Cor Raça Etnia')": 'flg_experiencia_profissional_prejudicada_raca',
"('P1_e_3 ', 'Experiencia prejudicada devido a minha identidade de gênero')": 'flg_experiencia_profissional_prejudicada_genero',
"('P1_e_4 ', 'Experiencia prejudicada devido ao fato de ser PCD')": 'flg_experiencia_profissional_prejudicada_pcd',
"('P1_f ', 'aspectos_prejudicados')": 'dsc_P1f_aspectos_prejudicados',
"('P1_f_1', 'Quantidade de oportunidades de emprego/vagas recebidas')": 'flg_P1f_oportunidades_emprego_recebidas',
"('P1_f_2', 'Senioridade das vagas recebidas em relação à sua experiência')": 'flg_P2f_senioridade_vagas_recebidas',
"('P1_f_3', 'Aprovação em processos seletivos/entrevistas')": 'flg_P3f_aprovacao_processos_seletivos',
"('P1_f_4', 'Oportunidades de progressão de carreira')": 'flg_P4f_oportunidades_progressao_carreira',
"('P1_f_5', 'Velocidade de progressão de carreira')": 'flg_P5f_velocidade_progressao_carreira',
"('P1_f_6', 'Nível de cobrança no trabalho/Stress no trabalho')": 'flg_P6f_nivel_cobranca_stress_trabalho',
"('P1_f_7', 'Atenção dada diante das minhas opiniões e ideias')": 'flg_P7f_atencao_ideias',
"('P1_f_8', 'Relação com outros membros da empresa, em momentos de trabalho')": 'flg_P8f_relacao_membros_trabalho',
"('P1_f_9', 'Relação com outros membros da empresa, em momentos de integração e outros momentos fora do trabalho')": 'flg_P9f_relacao_membros_fora_trabalho',
"('P1_i ', 'Estado onde mora')": 'dsc_estado',
"('P1_l ', 'Nivel de Ensino')": 'dsc_nivel_ensino',
"('P1_m ', 'Área de Formação')": 'dsc_area_formacao',
"('P2_a ', 'Qual sua situação atual de trabalho?')": 'dsc_situacao_atual_trabalho',
"('P2_g ', 'Nivel')": 'dsc_nivel',
"('P2_h ', 'Faixa salarial')": 'dsc_faixa_salarial', #
"('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')": 'dsc_tempo_experiencia_dados',
"('P2_r ', 'Atualmente qual a sua forma de trabalho?')": 'dsc_forma_trabalho',
"('P4_d ', 'Quais das linguagens listadas abaixo você utiliza no trabalho?')": 'flg_P4d_linguagem_utilizada_trabalho',
"('P4_d_1 ', 'SQL')": 'flg_P4d_SQL',
"('P4_d_2 ', 'R ')": 'flg_P4d_R',
"('P4_d_3 ', 'Python')": 'flg_P4d_python',
"('P4_d_4 ', 'C/C++/C#')": 'flg_P4d_C',
"('P4_d_5 ', '.NET')": 'flg_P4d_NET',
"('P4_d_6 ', 'Java')": 'flg_P4d_java',
"('P4_d_7 ', 'Julia')": 'flg_P4d_julia',
"('P4_d_8 ', 'SAS/Stata')": 'flg_P4d_SAS',
"('P4_d_9 ', 'Visual Basic/VBA')": 'flg_P4d_VBA',
"('P4_d_10 ', 'Scala')": 'flg_P4d_scala',
"('P4_d_11 ', 'Matlab')": 'flg_P4d_matlab',
"('P4_d_12 ', 'Rust')": 'flg_P4d_rust',
"('P4_d_13 ', 'PHP')": 'flg_P4d_PHP',
"('P4_d_14 ', 'JavaScript')": 'flg_P4d_JavaScript',
"('P4_d_15 ', 'Não utilizo nenhuma linguagem')": 'flg_P4d_nenhuma',
"('P4_g ', 'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?')": 'flg_P4g_banco_utiliza',
"('P4_g_1 ', 'MySQL')": 'flg_P4g_MySQL',
"('P4_g_2 ', 'Oracle')": 'flg_P4g_Oracle',
"('P4_g_3 ', 'SQL SERVER')": 'flg_P4g_SQL_SERVER',
"('P4_g_4 ', 'Amazon Aurora ou RDS')": 'flg_P4g_RDS',
"('P4_g_5 ', 'DynamoDB')": 'flg_P4g_DynamoDB',
"('P4_g_6 ', 'CoachDB')": 'flg_P4g_CoachDB',
"('P4_g_7 ', 'Cassandra')": 'flg_P4g_Cassandra',
"('P4_g_8 ', 'MongoDB')": 'flg_P4g_MongoDB',
"('P4_g_9 ', 'MariaDB')": 'flg_P4g_MariaDB',
"('P4_g_10 ', 'Datomic')": 'flg_P4g_Datomic',
"('P4_g_11 ', 'S3')": 'flg_P4g_S3',
"('P4_g_12 ', 'PostgreSQL')": 'flg_P4g_PostgreSQL',
"('P4_g_13 ', 'ElasticSearch')": 'flg_P4g_ElasticSearch',
"('P4_g_14 ', 'DB2')": 'flg_P4g_DB2',
"('P4_g_15 ', 'Microsoft Access')": 'flg_P4g_Microsoft_Access',
"('P4_g_16 ', 'SQLite')": 'flg_P4g_SQLite',
"('P4_g_17 ', 'Sybase')": 'flg_P4g_Sybase',
"('P4_g_18 ', 'Firebase')": 'flg_P4g_Firebase',
"('P4_g_19 ', 'Vertica')": 'flg_P4g_Vertica',
"('P4_g_20 ', 'Redis')": 'flg_P4g_Redis',
"('P4_g_21 ', 'Neo4J')": 'flg_P4g_Neo4J',
"('P4_g_22 ', 'Google BigQuery')": 'flg_P4g_Google_BigQuery',
"('P4_g_23 ', 'Google Firestore')": 'flg_P4g_Google_Firestore',
"('P4_g_24 ', 'Amazon Redshift')": 'flg_P4g_Amazon_Redshift',
"('P4_g_25 ', 'Amazon Athena')": 'flg_P4g_Amazon_Athena',
"('P4_g_26 ', 'Snowflake')": 'flg_P4g_Snowflake',
"('P4_g_27 ', 'Databricks')": 'flg_P4g_Databricks',
"('P4_g_28 ', 'HBase')": 'flg_P4g_HBase',
"('P4_g_29 ', 'Presto')": 'flg_P4g_Presto',
"('P4_g_30 ', 'Splunk')": 'flg_P4g_Splunk',
"('P4_g_31 ', 'SAP HANA')": 'flg_P4g_SAP_HANA',
"('P4_g_32 ', 'Hive')": 'flg_P4g_Hive',
"('P4_g_33 ', 'Firebird')": 'flg_P4g_Firebird',
"('P4_m ', 'Utiliza ChatGPT ou LLMs no trabalho?')": 'flg_P4m_utiliza_ia_generativa',
"('P4_m_1 ', 'Não uso soluções de AI Generativa com foco em produtividade')": 'flg_P4m_nao_utiliza',
"('P4_m_2 ', 'Uso soluções gratuitas de AI Generativa com foco em produtividade')": 'flg_P4m_utiliza_ia_foco_produtividade',
"('P4_m_3 ', 'Uso e pago pelas soluções de AI Generativa com foco em produtividade')": 'flg_P4m_utiliza_e_paga_ia_foco_produtividade',
"('P4_m_4 ', 'A empresa que trabalho paga pelas soluções de AI Generativa com foco em produtividade')": 'flg_P4m_empresa_paga_ia_foco_produtividade',
"('P4_m_5 ', 'Uso soluções do tipo Copilot')": 'flg_P4m_utiliza_tipo_copilot',
"('P6_c ', 'Sua organização possui um Data Lake?')": 'P6c_empresa_possui_datalake',
"('P6_e ', 'Sua organização possui um Data Warehouse?')": 'P6c_empresa_possui_datawarehouse'


}
df_pandas_csv_renomeado = df_pandas_csv.rename(columns=colunas_renomeadas)


In [9]:
from pyspark.sql import SparkSession
# Renomeando as colunas do DataFrame

spark = SparkSession.builder.appName("csv_dados").getOrCreate()

df_spark_csv = spark.createDataFrame(df_pandas_csv_renomeado)
df_spark_csv.createOrReplaceTempView("dados_csv")



resultado_csv = spark.sql("""

  select
          id,
          dsc_idade,
          dsc_faixa_idade,
          dsc_genero,
          dsc_cor_raca_etnia,
          dsc_pcd,
          dsc_experiencia_profissional_prejudicada,
          flg_experiencia_profissional_nao_prejudicada,
          flg_experiencia_profissional_prejudicada_raca,
          flg_experiencia_profissional_prejudicada_genero,
          flg_experiencia_profissional_prejudicada_pcd,
          dsc_P1f_aspectos_prejudicados,
          flg_P1f_oportunidades_emprego_recebidas,
          flg_P2f_senioridade_vagas_recebidas,
          flg_P3f_aprovacao_processos_seletivos,
          flg_P4f_oportunidades_progressao_carreira,
          flg_P5f_velocidade_progressao_carreira,
          flg_P6f_nivel_cobranca_stress_trabalho,
          flg_P7f_atencao_ideias,
          flg_P8f_relacao_membros_trabalho,
          flg_P9f_relacao_membros_fora_trabalho,
          dsc_estado,
          dsc_nivel_ensino,
          dsc_area_formacao,
          dsc_situacao_atual_trabalho,
          dsc_nivel,
          dsc_faixa_salarial,
          dsc_tempo_experiencia_dados,
          dsc_forma_trabalho,
          flg_P4d_linguagem_utilizada_trabalho,
          flg_P4d_SQL,
          flg_P4d_R,
          flg_P4d_python,
          flg_P4d_C,
          flg_P4d_NET,
          flg_P4d_java,
          flg_P4d_julia,
          flg_P4d_SAS,
          flg_P4d_VBA,
          flg_P4d_scala,
          flg_P4d_matlab,
          flg_P4d_rust,
          flg_P4d_PHP,
          flg_P4d_JavaScript,
          flg_P4d_nenhuma,
          flg_P4g_banco_utiliza,
          flg_P4g_MySQL,
          flg_P4g_Oracle,
          flg_P4g_SQL_SERVER,
          flg_P4g_RDS,
          flg_P4g_DynamoDB,
          flg_P4g_CoachDB,
          flg_P4g_Cassandra,
          flg_P4g_MongoDB,
          flg_P4g_MariaDB,
          flg_P4g_Datomic,
          flg_P4g_S3,
          flg_P4g_PostgreSQL,
          flg_P4g_ElasticSearch,
          flg_P4g_DB2,
          flg_P4g_Microsoft_Access,
          flg_P4g_SQLite,
          flg_P4g_Sybase,
          flg_P4g_Firebase,
          flg_P4g_Vertica,
          flg_P4g_Redis,
          flg_P4g_Neo4J,
          flg_P4g_Google_BigQuery,
          flg_P4g_Google_Firestore,
          flg_P4g_Amazon_Redshift,
          flg_P4g_Amazon_Athena,
          flg_P4g_Snowflake,
          flg_P4g_Databricks,
          flg_P4g_HBase,
          flg_P4g_Presto,
          flg_P4g_Splunk,
          flg_P4g_SAP_HANA,
          flg_P4g_Hive,
          flg_P4g_Firebird,
          flg_P4m_utiliza_ia_generativa,
          flg_P4m_nao_utiliza,
          flg_P4m_utiliza_ia_foco_produtividade,
          flg_P4m_utiliza_e_paga_ia_foco_produtividade,
          flg_P4m_empresa_paga_ia_foco_produtividade,
          flg_P4m_utiliza_tipo_copilot,
          P6c_empresa_possui_datalake,
          P6c_empresa_possui_datawarehouse

    from  dados_csv
""")

df_dados_2023 = resultado_csv.toPandas()

print("Dados do CSV processados e convertidos de volta para Pandas:")
display(df_dados_2023)



Dados do CSV processados e convertidos de volta para Pandas:


,id,dsc_idade,dsc_faixa_idade,dsc_genero,dsc_cor_raca_etnia,dsc_pcd,dsc_experiencia_profissional_prejudicada,flg_experiencia_profissional_nao_prejudicada,flg_experiencia_profissional_prejudicada_raca,flg_experiencia_profissional_prejudicada_genero,...,flg_P4g_Hive,flg_P4g_Firebird,flg_P4m_utiliza_ia_generativa,flg_P4m_nao_utiliza,flg_P4m_utiliza_ia_foco_produtividade,flg_P4m_utiliza_e_paga_ia_foco_produtividade,flg_P4m_empresa_paga_ia_foco_produtividade,flg_P4m_utiliza_tipo_copilot,P6c_empresa_possui_datalake,P6c_empresa_possui_datawarehouse
0,001b2d1qtli8t9z7oqgdhj001b2d4i0g,31,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,1.0,0.0,Não utilizo nenhum tipo de solução de IA Gener...,1.0,0.0,0.0,0.0,0.0,NaN,NaN
1,0026aa3fwd78u0026asg7456tfkjg2cs,30,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,0.0,0.0,Utilizo soluções pagas de AI Generativa (como ...,0.0,0.0,1.0,0.0,0.0,NaN,NaN
2,00r21rb9pusd1b0v7ew00r21rw3dy69w,37,35-39,Feminino,Amarela,Não,"Sim, acredito que a minha a experiência profis...",0.0,0.0,1.0,...,1.0,0.0,Utilizo apenas soluções gratuitas (como por ex...,0.0,1.0,0.0,0.0,0.0,NaN,NaN
3,00urm3jf2cek12w6ygue00urm3jzd17j,22,22-24,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00v0az4g792svil00vn6y1kfm9hq8vy9,34,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,0.0,0.0,Utilizo apenas soluções gratuitas (como por ex...,0.0,1.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,zyg9519671iqrgy93qzyg95183k8ix5f,34,30-34,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5289,zyppgxzxko25qkgwsbrgpzyppgxzxs81,32,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,0.0,0.0,Utilizo soluções pagas de AI Generativa (como ...,0.0,0.0,0.0,1.0,1.0,NaN,NaN
5290,zysl5f50cnz3r0cbe6t2zysrxbzfsrng,25,25-29,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,0.0,0.0,Utilizo apenas soluções gratuitas (como por ex...,0.0,1.0,0.0,0.0,0.0,NaN,NaN
5291,zyt2656c9e1becxp457zyt26566dsng8,31,30-34,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# from pyspark.sql import SparkSession
# # Renomeando as colunas do DataFrame

# spark = SparkSession.builder.appName("csv_dados").getOrCreate()

# df_spark_csv = spark.createDataFrame(df_pandas_csv_renomeado)
# df_spark_csv.createOrReplaceTempView("dados_csv")

# resultado_csv = spark.sql("""

#   select
#           id,
#           dsc_idade,
#           dsc_faixa_idade,
#           dac_genero,
#           dsc_cor_raca_etnia,
#           dsc_pcd,
#           dsc_experiencia_profissional_prejudicada,
#           flg_experiencia_profissional_nao_prejudicada,
#           flg_experiencia_profissional_prejudicada_raca,
#           flg_experiencia_profissional_prejudicada_genero,
#           flg_experiencia_profissional_prejudicada_pcd,
#           dsc_P1f_aspectos_prejudicados,
#           flg_P1f_oportunidades_emprego_recebidas,
#           flg_P2f_senioridade_vagas_recebidas,
#           flg_P3f_aprovacao_processos_seletivos,
#           flg_P4f_oportunidades_progressao_carreira,
#           flg_P5f_velocidade_progressao_carreira,
#           flg_P6f_nivel_cobranca_stress_trabalho,
#           flg_P7f_atencao_ideias,
#           flg_P8f_relacao_membros_trabalho,
#           flg_P9f_relacao_membros_fora_trabalho,
#           dsc_estado,
#           dsc_nivel_ensino,
#           dsc_area_formacao,
#           dsc_situacao_atual_trabalho,
#           dsc_nivel,
#           dsc_faixa_salarial,
#           dsc_tempo_experiencia_dados,
#           dsc_forma_trabalho

#     from  dados_csv
# """)

# df_dados_diversidade = resultado_csv.toPandas()

# print("Dados do CSV processados e convertidos de volta para Pandas:")
# display(df_dados_diversidade)



Dados do CSV processados e convertidos de volta para Pandas:


,id,dsc_idade,dsc_faixa_idade,dac_genero,dsc_cor_raca_etnia,dsc_pcd,dsc_experiencia_profissional_prejudicada,flg_experiencia_profissional_nao_prejudicada,flg_experiencia_profissional_prejudicada_raca,flg_experiencia_profissional_prejudicada_genero,...,flg_P8f_relacao_membros_trabalho,flg_P9f_relacao_membros_fora_trabalho,dsc_estado,dsc_nivel_ensino,dsc_area_formacao,dsc_situacao_atual_trabalho,dsc_nivel,dsc_faixa_salarial,dsc_tempo_experiencia_dados,dsc_forma_trabalho
0,001b2d1qtli8t9z7oqgdhj001b2d4i0g,31,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,Minas Gerais (MG),Doutorado ou Phd,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Sênior,de R$ 12.001/mês a R$ 16.000/mês,de 3 a 4 anos,Modelo híbrido flexível (o funcionário tem lib...
1,0026aa3fwd78u0026asg7456tfkjg2cs,30,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,Espírito Santo (ES),Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empreendedor ou Empregado (CNPJ),Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 3 a 4 anos,Modelo 100% remoto
2,00r21rb9pusd1b0v7ew00r21rw3dy69w,37,35-39,Feminino,Amarela,Não,"Sim, acredito que a minha a experiência profis...",0.0,0.0,1.0,...,0.0,0.0,São Paulo (SP),Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Júnior,de R$ 6.001/mês a R$ 8.000/mês,de 1 a 2 anos,Modelo híbrido flexível (o funcionário tem lib...
3,00urm3jf2cek12w6ygue00urm3jzd17j,22,22-24,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,São Paulo (SP),Estudante de Graduação,Computação / Engenharia de Software / Sistemas...,"Desempregado, buscando recolocação",NaN,NaN,NaN,NaN
4,00v0az4g792svil00vn6y1kfm9hq8vy9,34,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,Minas Gerais (MG),Doutorado ou Phd,Economia/ Administração / Contabilidade / Fina...,Empreendedor ou Empregado (CNPJ),Júnior,de R$ 4.001/mês a R$ 6.000/mês,Menos de 1 ano,Modelo 100% remoto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,zyg9519671iqrgy93qzyg95183k8ix5f,34,30-34,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,Minas Gerais (MG),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Empreendedor ou Empregado (CNPJ),NaN,de R$ 20.001/mês a R$ 25.000/mês,de 7 a 10 anos,Modelo 100% remoto
5289,zyppgxzxko25qkgwsbrgpzyppgxzxs81,32,30-34,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,Rio Grande do Norte (RN),Graduação/Bacharelado,Outras Engenharias,Empregado (CLT),Pleno,de R$ 8.001/mês a R$ 12.000/mês,de 4 a 6 anos,Modelo 100% remoto
5290,zysl5f50cnz3r0cbe6t2zysrxbzfsrng,25,25-29,Masculino,Branca,Não,NaN,NaN,NaN,NaN,...,NaN,NaN,São Paulo (SP),Graduação/Bacharelado,Marketing / Publicidade / Comunicação / Jornal...,Empregado (CLT),Sênior,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,Modelo híbrido flexível (o funcionário tem lib...
5291,zyt2656c9e1becxp457zyt26566dsng8,31,30-34,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,1.0,0.0,0.0,...,NaN,NaN,São Paulo (SP),Estudante de Graduação,Computação / Engenharia de Software / Sistemas...,"Desempregado, buscando recolocação",NaN,NaN,NaN,NaN


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
import pandas as pd
import numpy as np

# Inicializando o Spark
spark = SparkSession.builder.appName("csv_dados").getOrCreate()

# Supondo que df_pandas_csv_renomeado já exista e tenha sido criado anteriormente
df_spark_csv = spark.createDataFrame(df_pandas_csv_renomeado)
df_spark_csv.createOrReplaceTempView("dados_csv")

# Definindo a função de conversão
# Ajuste na função de conversão para garantir que os retornos sejam inteiros
def converter_salario(salario):
    if pd.isna(salario):
        return np.nan  # ou 0, se preferir
    elif salario == "Menos de R$ 1.000/mês":
        return 500
    elif "de R$ 1.001/mês a R$ 2.000/mês" in salario:
        return 1500
    elif "de R$ 101/mês a R$ 2.000/mês" in salario:
        return 1050
    elif "de R$ 12.001/mês a R$ 16.000/mês" in salario:
        return 14000
    elif "de R$ 16.001/mês a R$ 20.000/mês" in salario:
        return 18000
    elif "de R$ 2.001/mês a R$ 3.000/mês" in salario:
        return 2500
    elif "de R$ 20.001/mês a R$ 25.000/mês" in salario:
        return 22500
    elif "de R$ 25.001/mês a R$ 30.000/mês" in salario:
        return 27500
    elif "de R$ 3.001/mês a R$ 4.000/mês" in salario:
        return 3500
    elif "de R$ 30.001/mês a R$ 40.000/mês" in salario:
        return 35000
    elif "de R$ 4.001/mês a R$ 6.000/mês" in salario:
        return 5000
    elif "de R$ 6.001/mês a R$ 8.000/mês" in salario:
        return 7000
    elif "de R$ 8.001/mês a R$ 12.000/mês" in salario:
        return 10000
    else:
        return np.nan

# Convertendo a coluna e garantindo que a nova coluna seja tratada como int
df_spark_csv = df_spark_csv.rdd.map(lambda row: row + (int(converter_salario(row.dsc_faixa_salarial)) if not pd.isna(converter_salario(row.dsc_faixa_salarial)) else None,)) \
    .toDF(df_spark_csv.columns + ['Salario_Convertido'])

# Salario_Convertido com valores inteiros explicitamente
df_spark_csv = df_spark_csv.withColumn("Salario_Convertido", col("Salario_Convertido").cast(IntegerType()))

# Criando a visualização temporária
df_spark_csv.createOrReplaceTempView("dados_csv")

# Realizando a consulta SQL, garantindo que a coluna Salario_Convertido seja tratada como int
resultado_csv = spark.sql("""
    SELECT
        id,
        dsc_idade,
        dsc_faixa_idade,
        dsc_genero,
        dsc_cor_raca_etnia,
        dsc_pcd,
        dsc_experiencia_profissional_prejudicada,
        CASE
            WHEN dsc_experiencia_profissional_prejudicada LIKE '%Sim%' THEN 'Experiência prejudicada'
            WHEN dsc_experiencia_profissional_prejudicada LIKE '%Não%' THEN 'Experiência não prejudicada'
            ELSE 'Não informado'
        END AS dsc_experiencia_profissional_prejudicada_geral,
        flg_experiencia_profissional_nao_prejudicada,
        flg_experiencia_profissional_prejudicada_raca,
        flg_experiencia_profissional_prejudicada_genero,
        flg_experiencia_profissional_prejudicada_pcd,
        dsc_P1f_aspectos_prejudicados,
        flg_P1f_oportunidades_emprego_recebidas,
        flg_P2f_senioridade_vagas_recebidas,
        flg_P3f_aprovacao_processos_seletivos,
        flg_P4f_oportunidades_progressao_carreira,
        flg_P5f_velocidade_progressao_carreira,
        flg_P6f_nivel_cobranca_stress_trabalho,
        flg_P7f_atencao_ideias,
        flg_P8f_relacao_membros_trabalho,
        flg_P9f_relacao_membros_fora_trabalho,
        dsc_estado,
        dsc_nivel_ensino,
        dsc_area_formacao,
        dsc_situacao_atual_trabalho,
        dsc_nivel,
        dsc_faixa_salarial,
        cast(Salario_Convertido as int) as Salario_Convertido,
        dsc_tempo_experiencia_dados,
        dsc_forma_trabalho,
        P6c_empresa_possui_datalake,
        P6c_empresa_possui_datawarehouse
    FROM dados_csv
""")

# Convertendo o resultado de volta para Pandas
df_dados_diversidade = resultado_csv.toPandas()

df_spark_diversidade = spark.createDataFrame(df_dados_diversidade)
df_spark_diversidade.createOrReplaceTempView("dados_diversidade")
# df_dados_diversidade.to_csv('dados_diversidade_2023.csv', index=False)
print("Dados do CSV processados e convertidos de volta para Pandas:")
display(df_dados_diversidade)


Dados do CSV processados e convertidos de volta para Pandas:


,id,dsc_idade,dsc_faixa_idade,dsc_genero,dsc_cor_raca_etnia,dsc_pcd,dsc_experiencia_profissional_prejudicada,dsc_experiencia_profissional_prejudicada_geral,flg_experiencia_profissional_nao_prejudicada,flg_experiencia_profissional_prejudicada_raca,...,dsc_nivel_ensino,dsc_area_formacao,dsc_situacao_atual_trabalho,dsc_nivel,dsc_faixa_salarial,Salario_Convertido,dsc_tempo_experiencia_dados,dsc_forma_trabalho,P6c_empresa_possui_datalake,P6c_empresa_possui_datawarehouse
0,001b2d1qtli8t9z7oqgdhj001b2d4i0g,31,30-34,Masculino,Branca,Não,NaN,Não informado,NaN,NaN,...,Doutorado ou Phd,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Sênior,de R$ 12.001/mês a R$ 16.000/mês,14000.0,de 3 a 4 anos,Modelo híbrido flexível (o funcionário tem lib...,NaN,NaN
1,0026aa3fwd78u0026asg7456tfkjg2cs,30,30-34,Masculino,Branca,Não,NaN,Não informado,NaN,NaN,...,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empreendedor ou Empregado (CNPJ),Pleno,de R$ 6.001/mês a R$ 8.000/mês,7000.0,de 3 a 4 anos,Modelo 100% remoto,NaN,NaN
2,00r21rb9pusd1b0v7ew00r21rw3dy69w,37,35-39,Feminino,Amarela,Não,"Sim, acredito que a minha a experiência profis...",Experiência prejudicada,0.0,0.0,...,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Júnior,de R$ 6.001/mês a R$ 8.000/mês,7000.0,de 1 a 2 anos,Modelo híbrido flexível (o funcionário tem lib...,NaN,NaN
3,00urm3jf2cek12w6ygue00urm3jzd17j,22,22-24,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,Experiência não prejudicada,1.0,0.0,...,Estudante de Graduação,Computação / Engenharia de Software / Sistemas...,"Desempregado, buscando recolocação",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00v0az4g792svil00vn6y1kfm9hq8vy9,34,30-34,Masculino,Branca,Não,NaN,Não informado,NaN,NaN,...,Doutorado ou Phd,Economia/ Administração / Contabilidade / Fina...,Empreendedor ou Empregado (CNPJ),Júnior,de R$ 4.001/mês a R$ 6.000/mês,5000.0,Menos de 1 ano,Modelo 100% remoto,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,zyg9519671iqrgy93qzyg95183k8ix5f,34,30-34,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,Experiência não prejudicada,1.0,0.0,...,Pós-graduação,Computação / Engenharia de Software / Sistemas...,Empreendedor ou Empregado (CNPJ),NaN,de R$ 20.001/mês a R$ 25.000/mês,22500.0,de 7 a 10 anos,Modelo 100% remoto,NaN,NaN
5289,zyppgxzxko25qkgwsbrgpzyppgxzxs81,32,30-34,Masculino,Branca,Não,NaN,Não informado,NaN,NaN,...,Graduação/Bacharelado,Outras Engenharias,Empregado (CLT),Pleno,de R$ 8.001/mês a R$ 12.000/mês,10000.0,de 4 a 6 anos,Modelo 100% remoto,NaN,NaN
5290,zysl5f50cnz3r0cbe6t2zysrxbzfsrng,25,25-29,Masculino,Branca,Não,NaN,Não informado,NaN,NaN,...,Graduação/Bacharelado,Marketing / Publicidade / Comunicação / Jornal...,Empregado (CLT),Sênior,de R$ 8.001/mês a R$ 12.000/mês,10000.0,de 1 a 2 anos,Modelo híbrido flexível (o funcionário tem lib...,NaN,NaN
5291,zyt2656c9e1becxp457zyt26566dsng8,31,30-34,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,Experiência não prejudicada,1.0,0.0,...,Estudante de Graduação,Computação / Engenharia de Software / Sistemas...,"Desempregado, buscando recolocação",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, concat_ws, col

# Inicializando o Spark
spark = SparkSession.builder.appName("csv_dados").getOrCreate()
df_spark_csv = spark.createDataFrame(df_pandas_csv_renomeado)
df_spark_csv.createOrReplaceTempView("dados_csv2")

# mudando as flags para texto
resultado_csv = spark.sql("""
    SELECT
        id as id_linguagem,
        flg_P4d_linguagem_utilizada_trabalho,
        CASE WHEN flg_P4d_SQL = 1 THEN 'SQL' ELSE '' END AS dsc_P4d_SQL,
        CASE WHEN flg_P4d_R = 1 THEN 'R' ELSE '' END AS dsc_P4d_R,
        CASE WHEN flg_P4d_python = 1 THEN 'Python' ELSE '' END AS dsc_P4d_python,
        CASE WHEN flg_P4d_C = 1 THEN 'C/C++/C#' ELSE '' END AS dsc_P4d_C,
        CASE WHEN flg_P4d_NET = 1 THEN '.NET' ELSE '' END AS dsc_P4d_NET,
        CASE WHEN flg_P4d_java = 1 THEN 'Java' ELSE '' END AS dsc_P4d_java,
        CASE WHEN flg_P4d_julia = 1 THEN 'Julia' ELSE '' END AS dsc_P4d_julia,
        CASE WHEN flg_P4d_SAS = 1 THEN 'SAS' ELSE '' END AS dsc_P4d_SAS,
        CASE WHEN flg_P4d_VBA = 1 THEN 'Visual Basic/VBA' ELSE '' END AS dsc_P4d_VBA,
        CASE WHEN flg_P4d_scala = 1 THEN 'Scala' ELSE '' END AS dsc_P4d_scala,
        CASE WHEN flg_P4d_matlab = 1 THEN 'Matlab' ELSE '' END AS dsc_P4d_matlab,
        CASE WHEN flg_P4d_rust = 1 THEN 'Rust' ELSE '' END AS dsc_P4d_rust,
        CASE WHEN flg_P4d_PHP = 1 THEN 'PHP' ELSE '' END AS dsc_P4d_PHP,
        CASE WHEN flg_P4d_JavaScript = 1 THEN 'JavaScript' ELSE '' END AS dsc_P4d_JavaScript,
        CASE WHEN flg_P4d_nenhuma = 1 THEN 'Nenhuma' ELSE '' END AS dsc_P4d_nenhuma
    FROM dados_csv2
""")

# Concatenando as linguagens utilizadas em uma única coluna
resultado_final = resultado_csv.withColumn(
    "dsc_linguagem_utilizada_trabalho",
    concat_ws(", ",
        when(col("dsc_P4d_SQL") != '', col("dsc_P4d_SQL")),
        when(col("dsc_P4d_R") != '', col("dsc_P4d_R")),
        when(col("dsc_P4d_python") != '', col("dsc_P4d_python")),
        when(col("dsc_P4d_C") != '', col("dsc_P4d_C")),
        when(col("dsc_P4d_NET") != '', col("dsc_P4d_NET")),
        when(col("dsc_P4d_java") != '', col("dsc_P4d_java")),
        when(col("dsc_P4d_julia") != '', col("dsc_P4d_julia")),
        when(col("dsc_P4d_SAS") != '', col("dsc_P4d_SAS")),
        when(col("dsc_P4d_VBA") != '', col("dsc_P4d_VBA")),
        when(col("dsc_P4d_scala") != '', col("dsc_P4d_scala")),
        when(col("dsc_P4d_matlab") != '', col("dsc_P4d_matlab")),
        when(col("dsc_P4d_rust") != '', col("dsc_P4d_rust")),
        when(col("dsc_P4d_PHP") != '', col("dsc_P4d_PHP")),
        when(col("dsc_P4d_JavaScript") != '', col("dsc_P4d_JavaScript")),
        when(col("dsc_P4d_nenhuma") != '', col("dsc_P4d_nenhuma"))
    )
)
resultado_final_selecionado = resultado_final.select("id_linguagem", "dsc_linguagem_utilizada_trabalho")

# Convertendo o resultado de volta para um DataFrame Pandas
df_dados_linguagem = resultado_final_selecionado.toPandas()
df_spark_csv = spark.createDataFrame(df_dados_linguagem)
df_spark_csv.createOrReplaceTempView("dados_linguagem") # criando a base
print("Dados do CSV processados e convertidos de volta para Pandas:")
display(df_dados_linguagem)


Dados do CSV processados e convertidos de volta para Pandas:


,id_linguagem,dsc_linguagem_utilizada_trabalho
0,001b2d1qtli8t9z7oqgdhj001b2d4i0g,"SQL, Python"
1,0026aa3fwd78u0026asg7456tfkjg2cs,"SQL, Python"
2,00r21rb9pusd1b0v7ew00r21rw3dy69w,"SQL, Python, SAS"
3,00urm3jf2cek12w6ygue00urm3jzd17j,
4,00v0az4g792svil00vn6y1kfm9hq8vy9,"SQL, Python"
...,...,...
5288,zyg9519671iqrgy93qzyg95183k8ix5f,
5289,zyppgxzxko25qkgwsbrgpzyppgxzxs81,"SQL, Python"
5290,zysl5f50cnz3r0cbe6t2zysrxbzfsrng,"SQL, Python"
5291,zyt2656c9e1becxp457zyt26566dsng8,


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, concat_ws, col

# Inicializando o Spark
spark = SparkSession.builder.appName("csv_dados").getOrCreate()
df_spark_csv = spark.createDataFrame(df_pandas_csv_renomeado)
df_spark_csv.createOrReplaceTempView("dados_csv3")

# transformando as flags em texto para depois concatenar
resultado_csv = spark.sql("""
    SELECT
        id as id_banco,
        flg_P4g_banco_utiliza,
        CASE WHEN flg_P4g_MySQL = 1 THEN 'MySQL' ELSE '' END AS dsc_P4g_MySQL,
        CASE WHEN flg_P4g_Oracle = 1 THEN 'Oracle' ELSE '' END AS dsc_P4g_Oracle,
        CASE WHEN flg_P4g_SQL_SERVER = 1 THEN 'SQL Server' ELSE '' END AS dsc_P4g_SQL_SERVER,
        CASE WHEN flg_P4g_RDS = 1 THEN 'RDS' ELSE '' END AS dsc_P4g_RDS,
        CASE WHEN flg_P4g_DynamoDB = 1 THEN 'DynamoDB' ELSE '' END AS dsc_P4g_DynamoDB,
        CASE WHEN flg_P4g_CoachDB = 1 THEN 'CoachDB' ELSE '' END AS dsc_P4g_CoachDB,
        CASE WHEN flg_P4g_Cassandra = 1 THEN 'Cassandra' ELSE '' END AS dsc_P4g_Cassandra,
        CASE WHEN flg_P4g_MongoDB = 1 THEN 'MongoDB' ELSE '' END AS dsc_P4g_MongoDB,
        CASE WHEN flg_P4g_MariaDB = 1 THEN 'MariaDB' ELSE '' END AS dsc_P4g_MariaDB,
        CASE WHEN flg_P4g_Datomic = 1 THEN 'Datomic' ELSE '' END AS dsc_P4g_Datomic,
        CASE WHEN flg_P4g_S3 = 1 THEN 'S3' ELSE '' END AS dsc_P4g_S3,
        CASE WHEN flg_P4g_PostgreSQL = 1 THEN 'PostgreSQL' ELSE '' END AS dsc_P4g_PostgreSQL,
        CASE WHEN flg_P4g_ElasticSearch = 1 THEN 'ElasticSearch' ELSE '' END AS dsc_P4g_ElasticSearch,
        CASE WHEN flg_P4g_DB2 = 1 THEN 'DB2' ELSE '' END AS dsc_P4g_DB2,
        CASE WHEN flg_P4g_Microsoft_Access = 1 THEN 'Microsoft Access' ELSE '' END AS dsc_P4g_Microsoft_Access,
        CASE WHEN flg_P4g_SQLite = 1 THEN 'SQLite' ELSE '' END AS dsc_P4g_SQLite,
        CASE WHEN flg_P4g_Sybase = 1 THEN 'Sybase' ELSE '' END AS dsc_P4g_Sybase,
        CASE WHEN flg_P4g_Firebase = 1 THEN 'Firebase' ELSE '' END AS dsc_P4g_Firebase,
        CASE WHEN flg_P4g_Vertica = 1 THEN 'Vertica' ELSE '' END AS dsc_P4g_Vertica,
        CASE WHEN flg_P4g_Redis = 1 THEN 'Redis' ELSE '' END AS dsc_P4g_Redis,
        CASE WHEN flg_P4g_Neo4J = 1 THEN 'Neo4J' ELSE '' END AS dsc_P4g_Neo4J,
        CASE WHEN flg_P4g_Google_BigQuery = 1 THEN 'Google BigQuery' ELSE '' END AS dsc_P4g_Google_BigQuery,
        CASE WHEN flg_P4g_Google_Firestore = 1 THEN 'Google Firestore' ELSE '' END AS dsc_P4g_Google_Firestore,
        CASE WHEN flg_P4g_Amazon_Redshift = 1 THEN 'Amazon Redshift' ELSE '' END AS dsc_P4g_Amazon_Redshift,
        CASE WHEN flg_P4g_Amazon_Athena = 1 THEN 'Amazon Athena' ELSE '' END AS dsc_P4g_Amazon_Athena,
        CASE WHEN flg_P4g_Snowflake = 1 THEN 'Snowflake' ELSE '' END AS dsc_P4g_Snowflake,
        CASE WHEN flg_P4g_Databricks = 1 THEN 'Databricks' ELSE '' END AS dsc_P4g_Databricks,
        CASE WHEN flg_P4g_HBase = 1 THEN 'HBase' ELSE '' END AS dsc_P4g_HBase,
        CASE WHEN flg_P4g_Presto = 1 THEN 'Presto' ELSE '' END AS dsc_P4g_Presto,
        CASE WHEN flg_P4g_Splunk = 1 THEN 'Splunk' ELSE '' END AS dsc_P4g_Splunk,
        CASE WHEN flg_P4g_SAP_HANA = 1 THEN 'SAP HANA' ELSE '' END AS dsc_P4g_SAP_HANA,
        CASE WHEN flg_P4g_Hive = 1 THEN 'Hive' ELSE '' END AS dsc_P4g_Hive,
        CASE WHEN flg_P4g_Firebird = 1 THEN 'Firebird' ELSE '' END AS dsc_P4g_Firebird
    FROM dados_csv3
""")

# Concatenando as bases de dados utilizadas em uma única coluna
resultado_final = resultado_csv.withColumn(
    "dsc_banco_utiliza_trabalho",
    concat_ws(", ",
        when(col("dsc_P4g_MySQL") != '', col("dsc_P4g_MySQL")),
        when(col("dsc_P4g_Oracle") != '', col("dsc_P4g_Oracle")),
        when(col("dsc_P4g_SQL_SERVER") != '', col("dsc_P4g_SQL_SERVER")),
        when(col("dsc_P4g_RDS") != '', col("dsc_P4g_RDS")),
        when(col("dsc_P4g_DynamoDB") != '', col("dsc_P4g_DynamoDB")),
        when(col("dsc_P4g_CoachDB") != '', col("dsc_P4g_CoachDB")),
        when(col("dsc_P4g_Cassandra") != '', col("dsc_P4g_Cassandra")),
        when(col("dsc_P4g_MongoDB") != '', col("dsc_P4g_MongoDB")),
        when(col("dsc_P4g_MariaDB") != '', col("dsc_P4g_MariaDB")),
        when(col("dsc_P4g_Datomic") != '', col("dsc_P4g_Datomic")),
        when(col("dsc_P4g_S3") != '', col("dsc_P4g_S3")),
        when(col("dsc_P4g_PostgreSQL") != '', col("dsc_P4g_PostgreSQL")),
        when(col("dsc_P4g_ElasticSearch") != '', col("dsc_P4g_ElasticSearch")),
        when(col("dsc_P4g_DB2") != '', col("dsc_P4g_DB2")),
        when(col("dsc_P4g_Microsoft_Access") != '', col("dsc_P4g_Microsoft_Access")),
        when(col("dsc_P4g_SQLite") != '', col("dsc_P4g_SQLite")),
        when(col("dsc_P4g_Sybase") != '', col("dsc_P4g_Sybase")),
        when(col("dsc_P4g_Firebase") != '', col("dsc_P4g_Firebase")),
        when(col("dsc_P4g_Vertica") != '', col("dsc_P4g_Vertica")),
        when(col("dsc_P4g_Redis") != '', col("dsc_P4g_Redis")),
        when(col("dsc_P4g_Neo4J") != '', col("dsc_P4g_Neo4J")),
        when(col("dsc_P4g_Google_BigQuery") != '', col("dsc_P4g_Google_BigQuery")),
        when(col("dsc_P4g_Google_Firestore") != '', col("dsc_P4g_Google_Firestore")),
        when(col("dsc_P4g_Amazon_Redshift") != '', col("dsc_P4g_Amazon_Redshift")),
        when(col("dsc_P4g_Amazon_Athena") != '', col("dsc_P4g_Amazon_Athena")),
        when(col("dsc_P4g_Snowflake") != '', col("dsc_P4g_Snowflake")),
        when(col("dsc_P4g_Databricks") != '', col("dsc_P4g_Databricks")),
        when(col("dsc_P4g_HBase") != '', col("dsc_P4g_HBase")),
        when(col("dsc_P4g_Presto") != '', col("dsc_P4g_Presto")),
        when(col("dsc_P4g_Splunk") != '', col("dsc_P4g_Splunk")),
        when(col("dsc_P4g_SAP_HANA") != '', col("dsc_P4g_SAP_HANA")),
        when(col("dsc_P4g_Hive") != '', col("dsc_P4g_Hive")),
        when(col("dsc_P4g_Firebird") != '', col("dsc_P4g_Firebird"))
    )
)
resultado_final_selecionado = resultado_final.select("id_banco", "dsc_banco_utiliza_trabalho")

# Convertendo o resultado de volta para um DataFrame Pandas
df_dados_banco = resultado_final_selecionado.toPandas()
df_spark_csv = spark.createDataFrame(df_dados_banco)
df_spark_csv.createOrReplaceTempView("dados_banco_utilizado") #criando a base
# Exibindo os dados
print("Dados do CSV processados e convertidos de volta para Pandas:")
display(df_dados_banco)


Dados do CSV processados e convertidos de volta para Pandas:


,id_banco,dsc_banco_utiliza_trabalho
0,001b2d1qtli8t9z7oqgdhj001b2d4i0g,"SQL Server, S3, SQLite, Redis, Hive"
1,0026aa3fwd78u0026asg7456tfkjg2cs,S3
2,00r21rb9pusd1b0v7ew00r21rw3dy69w,"S3, Amazon Athena, Hive"
3,00urm3jf2cek12w6ygue00urm3jzd17j,
4,00v0az4g792svil00vn6y1kfm9hq8vy9,"Google BigQuery, Snowflake"
...,...,...
5288,zyg9519671iqrgy93qzyg95183k8ix5f,
5289,zyppgxzxko25qkgwsbrgpzyppgxzxs81,Google BigQuery
5290,zysl5f50cnz3r0cbe6t2zysrxbzfsrng,"MongoDB, S3, Amazon Athena"
5291,zyt2656c9e1becxp457zyt26566dsng8,


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, concat_ws, col
import pandas

# Inicializando o Spark
spark = SparkSession.builder.appName("csv_dados").getOrCreate()

df_spark_diversidade = spark.createDataFrame(df_dados_diversidade)
df_spark_diversidade.createOrReplaceTempView("dados_diversidade")
# display(df_dados_diversidade)

df_spark_linguagem = spark.createDataFrame(df_dados_linguagem)
df_spark_linguagem.createOrReplaceTempView("dados_linguagem")
# display(df_dados_linguagem)

df_spark_banco = spark.createDataFrame(df_dados_banco)
df_spark_banco.createOrReplaceTempView("dados_banco_utilizado")
# display(df_dados_banco)

resultado = spark.sql("""
    SELECT
        dados_diversidade.id,
        dados_diversidade.dsc_idade,
        dados_diversidade.dsc_faixa_idade,
        dados_diversidade.dsc_genero,
        dados_diversidade.dsc_cor_raca_etnia,
        dados_diversidade.dsc_pcd,
        dados_diversidade.dsc_experiencia_profissional_prejudicada,
        dados_diversidade.dsc_experiencia_profissional_prejudicada_geral,
        cast(dados_diversidade.flg_experiencia_profissional_nao_prejudicada as int ) as flg_experiencia_profissional_nao_prejudicada,
        cast(dados_diversidade.flg_experiencia_profissional_prejudicada_raca as int ) as flg_experiencia_profissional_prejudicada_raca,
        cast(dados_diversidade.flg_experiencia_profissional_prejudicada_genero as int ) as flg_experiencia_profissional_prejudicada_genero,
        cast(dados_diversidade.flg_experiencia_profissional_prejudicada_pcd as int ) as flg_experiencia_profissional_prejudicada_pcd,
        dados_diversidade.dsc_P1f_aspectos_prejudicados,
        cast(dados_diversidade.flg_P1f_oportunidades_emprego_recebidas as int ) as flg_P1f_oportunidades_emprego_recebidas,
        cast(dados_diversidade.flg_P2f_senioridade_vagas_recebidas as int ) as flg_P2f_senioridade_vagas_recebidas,
        cast(dados_diversidade.flg_P3f_aprovacao_processos_seletivos as int ) as flg_P3f_aprovacao_processos_seletivos,
        cast(dados_diversidade.flg_P4f_oportunidades_progressao_carreira as int ) as flg_P4f_oportunidades_progressao_carreira,
        cast(dados_diversidade.flg_P5f_velocidade_progressao_carreira as int ) as flg_P5f_velocidade_progressao_carreira,
        cast(dados_diversidade.flg_P6f_nivel_cobranca_stress_trabalho as int ) as flg_P6f_nivel_cobranca_stress_trabalho,
        cast(dados_diversidade.flg_P7f_atencao_ideias as int ) as flg_P7f_atencao_ideias,
        cast(dados_diversidade.flg_P8f_relacao_membros_trabalho as int ) as flg_P8f_relacao_membros_trabalho,
        cast(dados_diversidade.flg_P9f_relacao_membros_fora_trabalho as int ) as flg_P9f_relacao_membros_fora_trabalho,
        dados_diversidade.dsc_estado,
        dados_diversidade.dsc_nivel_ensino,
        dados_diversidade.dsc_area_formacao,
        dados_diversidade.dsc_situacao_atual_trabalho,
        dados_diversidade.dsc_nivel,
        dados_diversidade.dsc_faixa_salarial,
        cast(dados_diversidade.Salario_Convertido as int) as Salario_Convertido,
        dados_diversidade.dsc_tempo_experiencia_dados,
        dados_diversidade.dsc_forma_trabalho,
        dados_linguagem.dsc_linguagem_utilizada_trabalho,
        dados_banco_utilizado.dsc_banco_utiliza_trabalho
    FROM dados_diversidade
    INNER JOIN dados_linguagem
        ON dados_diversidade.id = dados_linguagem.id_linguagem
    INNER JOIN dados_banco_utilizado
        ON dados_linguagem.id_linguagem = dados_banco_utilizado.id_banco
""")

df_dados_pesquisa_2023 = resultado.toPandas()
df_dados_pesquisa_2023.to_csv('dados_pesquisa_2023.csv', index=False)
display(df_dados_pesquisa_2023)


,id,dsc_idade,dsc_faixa_idade,dsc_genero,dsc_cor_raca_etnia,dsc_pcd,dsc_experiencia_profissional_prejudicada,dsc_experiencia_profissional_prejudicada_geral,flg_experiencia_profissional_nao_prejudicada,flg_experiencia_profissional_prejudicada_raca,...,dsc_nivel_ensino,dsc_area_formacao,dsc_situacao_atual_trabalho,dsc_nivel,dsc_faixa_salarial,Salario_Convertido,dsc_tempo_experiencia_dados,dsc_forma_trabalho,dsc_linguagem_utilizada_trabalho,dsc_banco_utiliza_trabalho
0,2pm8um79eusf9dp22pm80sfswptyqgu0,36,35-39,Masculino,Parda,Não,"Sim, acredito que a minha a experiência profis...",Experiência prejudicada,0,1,...,Pós-graduação,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),NaN,de R$ 16.001/mês a R$ 20.000/mês,18000,de 7 a 10 anos,Modelo 100% remoto,,
1,55fg0h92m2e67kinqx55fg0hvp86dnzc,26,25-29,Masculino,Preta,Não,Não acredito que minha experiência profissiona...,Experiência não prejudicada,1,0,...,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Somente Estudante (pós-graduação),NaN,NaN,0,NaN,NaN,,
2,5vtf1bmwd5josacqawtsy5vtf1bv44lv,40,40-44,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,Experiência não prejudicada,1,0,...,Pós-graduação,Estatística/ Matemática / Matemática Computaci...,Empregado (CLT),Pleno,de R$ 16.001/mês a R$ 20.000/mês,18000,de 3 a 4 anos,Modelo 100% presencial,"SQL, Python","Oracle, SQL Server, SAP HANA"
3,6j58w5bubbsilbti68nk02fy6j58wvbl,27,25-29,Feminino,Parda,Não,"Sim, acredito que a minha a experiência profis...",Experiência prejudicada,0,1,...,Graduação/Bacharelado,Estatística/ Matemática / Matemática Computaci...,Empregado (CLT),Pleno,de R$ 8.001/mês a R$ 12.000/mês,10000,de 4 a 6 anos,Modelo 100% remoto,"SQL, R, Python","MySQL, PostgreSQL, Amazon Redshift"
4,7jyhajbc5qw7oj7u7jyhafb1yik585q7,33,30-34,Masculino,Branca,Não,NaN,Não informado,0,0,...,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Pleno,de R$ 4.001/mês a R$ 6.000/mês,5000,de 1 a 2 anos,Modelo híbrido flexível (o funcionário tem lib...,"SQL, Python","MySQL, RDS, PostgreSQL, Amazon Athena, Presto"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,x1dofasd8x8c6vmwgdvipx1dofask0yd,29,25-29,Masculino,Preta,Não,"Sim, acredito que a minha a experiência profis...",Experiência prejudicada,0,1,...,Não tenho graduação formal,NaN,Empregado (CLT),Júnior,de R$ 2.001/mês a R$ 3.000/mês,2500,Menos de 1 ano,Modelo híbrido com dias fixos de trabalho pres...,SQL,MySQL
5289,y51qoodkidtea4hu59ey51qooux3zp6k,36,35-39,Masculino,Branca,Não,NaN,Não informado,0,0,...,Mestrado,Economia/ Administração / Contabilidade / Fina...,Empreendedor ou Empregado (CNPJ),NaN,de R$ 16.001/mês a R$ 20.000/mês,18000,Mais de 10 anos,Modelo 100% remoto,,
5290,yb0tqlmdo1wtk8cg00yb0tqlmd4fzgth,27,25-29,Feminino,Branca,Não,"Sim, acredito que a minha a experiência profis...",Experiência prejudicada,0,0,...,Mestrado,Economia/ Administração / Contabilidade / Fina...,Empregado (CLT),Pleno,de R$ 6.001/mês a R$ 8.000/mês,7000,de 3 a 4 anos,Modelo 100% remoto,"SQL, Python","MongoDB, PostgreSQL, Google BigQuery"
5291,zuj8faf8zjiajxgsdnzuj8r07t1sj9bf,33,30-34,Masculino,Amarela,Não,NaN,Não informado,0,0,...,Pós-graduação,Outras Engenharias,Empreendedor ou Empregado (CNPJ),NaN,de R$ 20.001/mês a R$ 25.000/mês,22500,de 3 a 4 anos,Modelo híbrido flexível (o funcionário tem lib...,,
